<a href="https://colab.research.google.com/github/mpurkeypile-usd/AAI590-Final_Project/blob/main/AAI590_Final_Project_MPurkeypile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Matt Purkeypile  
AAI590 Capstone: Final Project  
August 2025  

# TODO: Title

## Data Cleaning

## Exploratory Data Analysis

## Model Design and Building

## Model Training

## Model Optimization

## Model Analysis